<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-19.-Dynamic-Attributes-and-Properties" data-toc-modified-id="Chapter-19.-Dynamic-Attributes-and-Properties-1">Chapter 19. Dynamic Attributes and Properties</a></span><ul class="toc-item"><li><span><a href="#Dynamic-attributes" data-toc-modified-id="Dynamic-attributes-1.1">Dynamic attributes</a></span><ul class="toc-item"><li><span><a href="#Default-attribute-access-using-keys" data-toc-modified-id="Default-attribute-access-using-keys-1.1.1">Default attribute access using keys</a></span></li><li><span><a href="#Dynamic-attribute-access-using-dot-notation" data-toc-modified-id="Dynamic-attribute-access-using-dot-notation-1.1.2">Dynamic attribute access using dot notation</a></span></li></ul></li><li><span><a href="#The-shelve-module" data-toc-modified-id="The-shelve-module-1.2">The <code>shelve</code> module</a></span></li><li><span><a href="#Properties" data-toc-modified-id="Properties-1.3">Properties</a></span><ul class="toc-item"><li><span><a href="#The-property-constructor" data-toc-modified-id="The-property-constructor-1.3.1">The property constructor</a></span></li><li><span><a href="#defining-property-without-decorators" data-toc-modified-id="defining-property-without-decorators-1.3.2">defining property without decorators</a></span></li><li><span><a href="#pitfalls-of-defining-property-without-decorators" data-toc-modified-id="pitfalls-of-defining-property-without-decorators-1.3.3">pitfalls of defining property without decorators</a></span></li><li><span><a href="#Docstrings-in-properties" data-toc-modified-id="Docstrings-in-properties-1.3.4">Docstrings in properties</a></span></li></ul></li><li><span><a href="#Property-Factories" data-toc-modified-id="Property-Factories-1.4">Property Factories</a></span><ul class="toc-item"><li><span><a href="#Property-deleters" data-toc-modified-id="Property-deleters-1.4.1">Property deleters</a></span></li></ul></li><li><span><a href="#Special-Attributes/Functions-that-Affect-Attribute-Handling" data-toc-modified-id="Special-Attributes/Functions-that-Affect-Attribute-Handling-1.5">Special Attributes/Functions that Affect Attribute Handling</a></span></li></ul></li></ul></div>

# Chapter 19. Dynamic Attributes and Properties

## Dynamic attributes
implement special methods such as `__getattr__` and `__setattr__` to evaluate attribute access using dot notation 

### Default attribute access using keys

In [24]:
# { "Schedule":
#   { "conferences": [{"serial": 115 }],
#     "events": [
#       { "serial": 34505,
#         "name": "Why Schools Don´t Use Open Source to Teach Programming",
#         "event_type": "40-minute conference session",
#         "time_start": "2014-07-23 11:30:00",
#         "time_stop": "2014-07-23 12:10:00",
#         "venue_serial": 1462,
#         "description": "Aside from the fact that high school programming...",
#         "website_url": "http://oscon.com/oscon2014/public/schedule/detail/34505",
#         "speakers": [157509],
#         "categories": ["Education"] }
#     ],
#     "speakers": [
#       { "serial": 157509,
#         "name": "Robert Lefkowitz",
#         "photo": null,
#         "url": "http://sharewave.com/",
#         "position": "CTO",
#         "affiliation": "Sharewave",
#         "twitter": "sharewaveteam",
#         "bio": "Robert ´r0ml´ Lefkowitz is the CTO at Sharewave, a startup..." }
#     ],
#     "venues": [
#       { "serial": 1462,
#         "name": "F151",
#         "category": "Conference Venues" }
#     ]
#   }
# }

from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)   
        with urlopen(URL) as remote, open(JSON, 'wb') as local:   
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)   
    
feed = load()
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [21]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key)) 

  1 conferences
494 events
357 speakers
 53 venues


In [22]:
feed['Schedule']['speakers'][-1]['name'] 

'Carina C. Zona'

In [23]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [24]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [25]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

### Dynamic attribute access using dot notation

In [22]:
from collections import abc
import keyword

class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            # allow keyword use in map keys
            if keyword.iskeyword(key):   
                key += '_'
            self.__data[key] = value 

    def __getattr__(self, name):   
        if hasattr(self.__data, name): # check if __data dict obj has this attribute
            return getattr(self.__data, name)   
        else:
            return FrozenJSON.build(self.__data[name]) # build attribute from name

    @classmethod
    def build(cls, obj):   
        if isinstance(obj, abc.Mapping):  # check if obj is a map 
            return cls(obj) #build FrozenJSON with it
        elif isinstance(obj, abc.MutableSequence):   # check if obj is a list
            return [cls.build(item) for item in obj] # build list recursively
        else:   
            return obj # return item as is

In [25]:
raw_feed = load()
feed = FrozenJSON(raw_feed) 
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [26]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [27]:
talk = feed.Schedule.events[40]
talk.name

'There *Will* Be Bugs'

In [28]:
talk.speakers

[3471, 5199]

keyword usage - append '_' to key

In [30]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [31]:
grad.class_

1982

invalid attributes still don't work though

In [32]:
x = FrozenJSON({'2be':'or not'})

In [33]:
x.2be

SyntaxError: invalid syntax (<ipython-input-33-8694215ab5bd>, line 1)

The `build` method that instantiates the appropriate type can be moved to `__new__`

In [74]:
class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

    def __new__(cls, arg):   
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)   
        elif isinstance(arg, abc.MutableSequence):   
            return [cls(item) for item in arg]
        else:
            return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])   

## The `shelve` module

a simple key-value object database backed by the dbm module, with these characteristics:
* shelve.Shelf subclasses abc.MutableMapping, so it provides the essential methods we expect of a mapping type
* shelve.Shelf provides a few other I/O management methods, like sync and close; it’s also a context manager.
* Keys and values are saved whenever a new value is assigned to a key.
* The keys must be strings.
* The values must be objects that the pickle module can handle.


Restructure feed from previous example with shelve module

In [7]:
from urllib.request import urlopen
import inspect
import os
import json
import warnings

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'
DB_NAME = 'data/schedule2_db'
CONFERENCE = 'conference.115'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())

    with open(JSON) as fp:
        return json.load(fp)


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __eq__(self, other):
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

class MissingDatabaseError(RuntimeError):
    """Raised when a database is required but was not set."""


class DbRecord(Record):

    __db = None

    @staticmethod
    def set_db(db):
        DbRecord.__db = db

    @staticmethod
    def get_db():
        return DbRecord.__db

    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise

    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

class Event(DbRecord):

    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key))
                                  for key in spkr_serials]
        return self._speaker_objs

    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, DbRecord)
        if inspect.isclass(cls) and issubclass(cls, DbRecord):
            factory = cls
        else:
            factory = DbRecord
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [8]:
import shelve
db = shelve.open(DB_NAME)
if CONFERENCE not in db:
         load_db(db)
db['speaker.3471']


<DbRecord serial='speaker.3471'>

In [11]:
DbRecord.set_db(db)
event = DbRecord.fetch('event.33950')
event

<Event 'There *Will* Be Bugs'>

In [12]:
event.venue

<DbRecord serial='venue.1449'>

In [13]:
event.venue.name

'Portland 251'

In [14]:
for spkr in event.speakers:   
    print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Ravenscroft
speaker.5199: Alex Martelli


## Properties
Properties are frequently used to enforce business rules by changing a public attribute into an attribute managed by a getter and setter without affecting client code, as the next section shows.


In [15]:
 class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

LineItem can have a negative price - bad!

In [22]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

In [23]:
raisins.weight = -20  # garbage in..
raisins.subtotal()  

-139.0

In [26]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight   
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property   
    def weight(self):   
        return self.__weight   

    @weight.setter   
    def weight(self, value):
        if value > 0:
            self.__weight = value   
        else:
            raise ValueError('value must be > 0')   

The new lineitem class throws if a negative weight is given and without changing the interface

In [28]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.weight = -20  # garbage in..
raisins.subtotal()  

ValueError: value must be > 0

In [29]:
walnuts = LineItem('walnuts', 0, 10.00)

ValueError: value must be > 0

### The property constructor

In [35]:
# all arguments are callables
property(fget=None, fset=None, fdel=None, doc=None) 

### defining property without decorators

In [34]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    def get_weight(self):   
        return self.__weight

    def set_weight(self, value):   
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

    # Build the property and assign it to a public class attribute.
    weight = property(get_weight, set_weight)   

### pitfalls of defining property without decorators

In [46]:
class Class:  # 
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [51]:
obj = Class()
vars(obj)   # returns the __dict__ of obj,  
            # showing it has no instance attributes.

{}

In [48]:
obj.data

'the class data attr'

In [49]:
Class.data

'the class data attr'

In [50]:
obj.data = 'bar'
vars(obj)

{'data': 'bar'}

In [41]:
# Now reading from obj.data retrieves the value of the 
# instance attribute. When read from the obj instance
obj.data

'bar'

In [42]:
Class.data

'the class data attr'

In [52]:
Class.prop

In [53]:
obj.prop

'the prop value'

In [54]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [56]:
# The property is not shadowed by an instance attribute.
obj.__dict__['prop'] = 'foo' 
vars(obj)

{'prop': 'foo'}

In [58]:
obj.prop

'the prop value'

In [61]:
# Overwriting Class.prop destroys the property object. WTF
Class.prop = 'baz'
obj.prop

'foo'

### Docstrings in properties
can be set with the doc attribute in the property definition

In [64]:
class Foo:

    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']

    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [66]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



In [68]:
help(Foo.bar)

Help on property:

    The bar attribute



## Property Factories
create a `quantity` property factory

In [70]:
def quantity(storage_name):   

    def qty_getter(instance):   
        return instance.__dict__[storage_name]   

    def qty_setter(instance, value):   
        if value > 0:
            instance.__dict__[storage_name] = value   
        else:
            raise ValueError('value must be > 0')

    return property(qty_getter, qty_setter)   

# The LineItem definition with the factory looks much better without the 
# noise of the getter/setters.
class LineItem:
    weight = quantity('weight')   
    price = quantity('price')   

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight   
        self.price = price

    def subtotal(self):
        return self.weight * self.price   


In [72]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [73]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

In [74]:
class BlackKnight:

    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]

    @property
    def member(self):
        print('next member is:')
        return self.members[0]

    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

### Property deleters
can be set with the `fdel` attribute in the property definition

In [75]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [76]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [77]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [78]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [79]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


## Special Attributes/Functions that Affect Attribute Handling

`__class__`
  
   * A reference to the object’s class (i.e., `obj.__class__` is the same as `type(obj)`
   
`__dict__`
   * A mapping that stores the writable attributes of an object or class.
   
`__slots__`

   * __slots__ is a tuple of strings naming the allowed 

`dir([object])`

   * Lists most attributes of the object.
   
`getattr(object, name[, default])`

   * Gets the attribute identified by the name string from the 
   
`hasattr(object, name)`

   * Returns True if the named attribute exists in the object, or can be somehow fetched through it (by inheritance, for example).
   
`setattr(object, name, value)`

   * Assigns the value to the named attribute of object
   
`vars([object])`

   * Returns the `__dict__` of object;
   
`__delattr__(self, name)`

   * Always called when there is an attempt to delete an attribute using the del statement
   
`__dir__(self)`

   * Called when dir is invoked on the object, to provide a listing of attributes; 
   
`__getattr__(self, name)`
   
   * Called only when an attempt to retrieve the named attribute fails, after the obj, Class, and its superclasses are searched. 
   
`__getattribute__(self, name)`

   * Always called when there is an attempt to retrieve the named attribute
   
`__setattr__(self, name, value)`

   * Always called when there is an attempt to set the named attribute.